In [8]:
import pandas as pd

df = pd.read_csv('large_amounts_of_data.csv')

df = df[['Hero ID','Team','Order','Lane','MatchID']]

# Count the number of duplicates for each unique MatchID
duplicates_count = df['MatchID'].value_counts()

# Print the count of duplicates for each unique MatchID
print(duplicates_count.to_frame())

            count
MatchID          
7425367208    720
7425288381    720
7425165105    720
7425319964    720
7425127995    710
...           ...
7426370420     10
7426365368     10
7426363325     10
7426307824     10
2               8

[139 rows x 1 columns]


In [34]:
df = pd.read_csv('large_amounts_of_data.csv')
# Keep only the first 10 consecutive rows of each match ID
df_cleaned = df.groupby('MatchID').head(10)

# Reset the index of the cleaned DataFrame
df_cleaned.reset_index(drop=True, inplace=True)

# Save the cleaned DataFrame to a CSV file
df_cleaned.to_csv('cleaned_data.csv', index=False)
clean = pd.read_csv('cleaned_data.csv')

real_clean= clean.groupby('MatchID').head(10)

# Reset the index of the cleaned DataFrame
real_clean.reset_index(drop=True, inplace=True)
real_clean.to_csv('cleaned_data.csv', index=False)
print(len(real_clean['MatchID'].value_counts()))

191


In [ ]:
from sklearn.linear_model import LogisticRegression

class DotaLanePredictor(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(n_inputs, n_outputs)

        
    def forward(self, x):